In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import GridSearchCV, cross_val_score, RandomizedSearchCV
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.decomposition import PCA
from torch.utils.data import TensorDataset, DataLoader
from scipy.stats import uniform, randint, loguniform
from sklearn import preprocessing

In [80]:
data_train = pd.read_csv("./train.csv")
data_test = pd.read_csv("./test.csv")
X_train = data_train.iloc[:, 6:]
X_test = data_test.iloc[:, 5:]


"""
data = pd.concat([data_train, data_test], axis=0)
X = data.iloc[:, 6:]
classifiers = data.iloc[:, 1:4]
enc = OneHotEncoder()
data_oh = enc.fit_transform(classifiers).toarray()
data_oh = pd.DataFrame(data_oh)

# Nombre de lignes dans data_train
n_train = len(data_train)

# Séparer data en data_train et data_test
data_train_oh = data_oh.iloc[:n_train, :].reset_index(drop=True)
data_test_oh = data_oh.iloc[n_train:, :].reset_index(drop=True)
"""


'\ndata = pd.concat([data_train, data_test], axis=0)\nX = data.iloc[:, 6:]\nclassifiers = data.iloc[:, 1:4]\nenc = OneHotEncoder()\ndata_oh = enc.fit_transform(classifiers).toarray()\ndata_oh = pd.DataFrame(data_oh)\n\n# Nombre de lignes dans data_train\nn_train = len(data_train)\n\n# Séparer data en data_train et data_test\ndata_train_oh = data_oh.iloc[:n_train, :].reset_index(drop=True)\ndata_test_oh = data_oh.iloc[n_train:, :].reset_index(drop=True)\n'

In [86]:
#np.sum(X_train, axis=1)
X_normalised = pd.DataFrame(preprocessing.normalize(X_train))
np.sum(X_normalised, axis=1)

0        4.275457
1        6.491709
2        5.203686
3       11.019652
4       10.888498
          ...    
1295    10.896534
1296     0.714299
1297    10.633639
1298     7.150106
1299     5.362656
Length: 1300, dtype: float64

In [88]:
standardizer = StandardScaler()
X_st = pd.DataFrame(standardizer.fit_transform(X_train))
np.sum(X_st, axis=1)

0       -47.242542
1       -25.612119
2       -34.833110
3       410.474762
4       321.051996
           ...    
1295     64.044146
1296    -54.127140
1297     22.617288
1298    -20.594557
1299    -33.917856
Length: 1300, dtype: float64

In [ ]:
#spectrum_filtered = pd.DataFrame(savgol_filter(spectrum, 7, 3, deriv = 2, axis = 0))
#spectrum_filtered_st = zscore(spectrum_filtered, axis = 1)

y = data_train["PURITY"]/100
seed = 43
"""
# Combine original data with PCA components
standardizer = StandardScaler()
X_st = standardizer.fit_transform(X_train)
"""
# Perform PCA to retain 95% of the variance
pca = PCA(42)
pca.fit(X_train)
X_pca = pca.transform(X_train)

X_normalised = preprocessing.normalize(X_pca)

X_train, X_valid, y_train, y_valid = train_test_split(X_normalised, y, test_size=0.2, random_state = 42)

# Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).clone().detach()
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).reshape(-1, 1).clone().detach()
X_valid_tensor = torch.tensor(X_valid, dtype=torch.float32).clone().detach()
y_valid_tensor = torch.tensor(y_valid.values, dtype=torch.float32).reshape(-1, 1).clone().detach()

In [ ]:
#X_st = standardizer.transform(X_test)
X_pca = pca.transform(X_st)
X_test_normalised = preprocessing.normalize(X_test.values)
X_test_tensor = torch.tensor(X_pca, dtype=torch.float32).clone().detach() 

In [42]:
import torch
import torch.nn as nn
from torch import Tensor

class MyLoss(nn.Module):  # Hérite de nn.Module
    def __init__(self, reduction: str = 'mean'):
        super(MyLoss, self).__init__()
        self.reduction = reduction

    def forward(self, output: Tensor, target: Tensor) -> Tensor:
        # Calcul de la perte personnalisée
        loss = torch.maximum(torch.zeros_like(output), (output - target) - 0.05) + \
               torch.maximum(torch.zeros_like(target), (target - output) - 0.05)

        # Applique la réduction spécifiée
        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        else:  # 'none'
            return loss

In [43]:
class CustomLoss(nn.Module):
    def __init__(self, threshold=5.0):
        """
        Custom loss function to penalize predictions with an absolute percentage error greater than a threshold.
        
        Args:
            threshold (float): The maximum percentage error allowed (in %).
        """
        super(CustomLoss, self).__init__()
        self.threshold = threshold

    def forward(self, y_pred, y_true):
        """
        Compute the custom loss.

        Args:
            y_pred (torch.Tensor): Predicted values.
            y_true (torch.Tensor): Ground truth values.

        Returns:
            torch.Tensor: The computed loss.
        """
        # Compute absolute percentage error
        percentage_error = torch.abs((y_pred - y_true) / y_true) * 100

        # Mask for errors greater than the threshold
        mask = (percentage_error > self.threshold).float()

        # Penalize errors that exceed the threshold
        loss = mask * percentage_error

        # Return the mean loss
        return loss.mean()

In [14]:
# Définir le modèle de réseau de neurones simple
class FeedForwardNN(nn.Module):

    def __init__(self, input_size, lin_layer_sizes,
                 outpout_size, lin_layer_dropouts, activation):
        
        super().__init__()
        
        if activation == 0:
            self.activation = nn.ReLU()
        elif activation == 1:
            self.activation = nn.SiLU()
        elif activation == 2:
            self.activation = nn.Tanh()
        elif activation == 3:
            self.activation = nn.LeakyReLU()

    
        # Linear Layers
        first_lin_layer = nn.Linear(input_size, lin_layer_sizes[0])
    
        self.lin_layers = nn.ModuleList([first_lin_layer] + [nn.Linear(lin_layer_sizes[i], lin_layer_sizes[i + 1]) for i in range(len(lin_layer_sizes) - 1)])
        
        #for lin_layer in self.lin_layers:
            #nn.init.kaiming_normal_(lin_layer.weight.data)
      
        # Output Layer
        self.outpout_layer = nn.Linear(lin_layer_sizes[-1], outpout_size)
        #nn.init.kaiming_normal_(self.outpout_layer.weight.data)
    
    
        # Dropout Layers
        self.dropout_layers = nn.ModuleList([nn.Dropout(rate) for rate,size in zip(lin_layer_dropouts,lin_layer_sizes)])

    def forward(self, x):
  
        for lin_layer, dropout_layer in zip(self.lin_layers, self.dropout_layers):

            x = lin_layer(x)
        
            x = self.activation(x)

            x = dropout_layer(x)
      
        x = self.outpout_layer(x)
        x = nn.Sigmoid()(x)
    
        return x

# Définir la classe NeuralNetRegressor
class NeuralNetRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, input_size, random_state, eta=0.001, max_epochs=100, batch=10, lin_layer_sizes = [50, 50],
                 outpout_size = 1, lin_layer_dropouts = [0.4, 0.4], activation = 0):
        self.input_size = input_size
        self.random_state = random_state
        self.eta = eta
        self.max_epochs = max_epochs
        self.batch = batch
        self.lin_layer_sizes = lin_layer_sizes
        self.outpout_size = outpout_size
        self.lin_layer_dropouts = lin_layer_dropouts
        self.activation = activation
        self.model = FeedForwardNN(input_size, lin_layer_sizes,
                 outpout_size, lin_layer_dropouts, activation)
        self.criterion = nn.L1Loss() 
    
    def fit(self, X, y, do_print=False):
        optimizer = optim.Adam(self.model.parameters(), lr=self.eta)
        X_tensor = torch.tensor(X).clone().detach().float()
        y_tensor = torch.tensor(y).clone().detach().float()
        dataset = TensorDataset(X_tensor, y_tensor)
        dataloader = DataLoader(dataset, batch_size=self.batch, shuffle=True)
        self.model.train()
        # Training loop
        for epoch in range(self.max_epochs):
            epoch_loss = 0.0
            for batch_X, batch_y in dataloader:
                optimizer.zero_grad()  # Reset gradients
                outputs = self.model(batch_X)  # Forward pass
                loss = self.criterion(outputs, batch_y)  # Compute loss
                loss.backward()  # Backward pass
                optimizer.step()  # Update parameters
                epoch_loss += loss.item()
            if do_print:
                print(f"Epoch {epoch+1}/{self.max_epochs}, Loss: {epoch_loss / len(dataloader)}")
        return self
    
    def predict(self, X):
        self.model.eval()
        X_tensor = torch.tensor(X, dtype=torch.float32)
        with torch.no_grad():
            outputs = self.model(X_tensor).flatten()
        return outputs.numpy()
    
    def parameters(self):
        return self.model.parameters()

In [17]:
from sklearn.metrics import make_scorer

def my_scorer(y_true, y_pred):
    y_true = y_true.detach().numpy()
    return np.mean(np.abs(y_pred-y_true)*100<=5)

custom_scorer = make_scorer(my_scorer, greater_is_better=True)

In [8]:
[[randint.rvs(32, 128) for _ in range(randint.rvs(1, 4))]]

[[114, 81]]

In [13]:
loguniform(1e-4, 1e-1).rvs()

0.005867127272781008

In [62]:
class RandomLayers:
    def __init__(self,min_layers, max_layers, min_nodes, max_nodes):
        self.min_nodes = min_nodes
        self.max_nodes = max_nodes
        self.min_layers = min_layers
        self.max_layers = max_layers

    def rvs(self, random_state=None):
        if random_state is not None:
            prev = randint.random_state
            randint.random_state = random_state 

        res = [[randint.rvs(self.min_nodes, self.max_nodes) for _ in range(randint.rvs(self.min_layers, self.max_layers))]]

        if random_state is not None:
            randint.random_state = prev

        return res


In [63]:
class RandomDropout:
    def __init__(self, max_layers,min_dropout,max_dropout):
        self.min_dropout = min_dropout
        self.max_dropout = max_dropout
        self.max_layers = max_layers

    def rvs(self, random_state=None):
        if random_state is not None:
            prev = uniform.random_state
            uniform.random_state = random_state 
        res = [[uniform.rvs(self.min_dropout, self.max_dropout) for _ in range(self.max_layers)]]

        if random_state is not None:
            uniform.random_state = prev

        return res

In [91]:
# Initialiser le modèle
np.random.seed(seed)
torch.manual_seed(seed)

input_size = X_train_tensor.shape[1]
net = NeuralNetRegressor(input_size=input_size, random_state=seed)

max_layers = 4

# Définir les paramètres pour GridSearch
params_dist = {
    'eta': loguniform(1e-4, 1e-1),
    'max_epochs': randint(200, 500),
    'batch': randint(32, 50),
    'lin_layer_sizes': RandomLayers(1,max_layers,32,128),  # Taille de 1 à 4 couches, entre 32 et 128 neurones par couche
    'lin_layer_dropouts': RandomDropout(max_layers,0.,1.),  # Dropout entre 0 et 1. pour chaque couche
    'activation': randint(0, 4),
}
# faire une classe qui genere les valeurs aleatoirement, et creer objet de la classe dans param dist pour lin lyer sizes et dropout faire un seule valeur pour toute les couches
# Initialiser RandomizedSearchCV
random_search = RandomizedSearchCV(net, params_dist, refit=True, cv=5, random_state=seed, scoring= 'neg_mean_absolute_error', verbose=0, n_iter= 5) 

# Entraîner le modèle avec GridSearch
random_grid_result = random_search.fit(X_train_tensor, y_train_tensor)
nouveau_model = random_grid_result.best_estimator_

print("Best MSE: %f using %s" % (random_grid_result.best_score_, random_grid_result.best_params_))

/var/folders/93/xgf_b79s5sd1vjn_794f14ym0000gn/T/ipykernel_48663/3560239249.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tensor = torch.tensor(X).clone().detach().float()
/var/folders/93/xgf_b79s5sd1vjn_794f14ym0000gn/T/ipykernel_48663/3560239249.py:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_tensor = torch.tensor(y).clone().detach().float()
/var/folders/93/xgf_b79s5sd1vjn_794f14ym0000gn/T/ipykernel_48663/3560239249.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tensor = torch.tensor(X, dtype=torch.float32)
/var/

Best MSE: -0.018972 using {'activation': 0, 'batch': 47, 'eta': 0.0017685051085424622, 'lin_layer_dropouts': [[0.6530056733559008, 0.6205716354509077, 0.4037700398666926, 0.8737659376448997]], 'lin_layer_sizes': [[105]], 'max_epochs': 335}


In [ ]:
y_pred = nouveau_model.predict(X_valid_tensor)
y_pred_train = nouveau_model.predict(X_train_tensor)
predictions = nouveau_model.predict(X_test_tensor)*100

# Vérifiez les sorties du modèle
print("y_pred contains NaN:", np.isnan(y_pred).any())


# Calculer la MSE
mse = np.mean(((y_pred - y_valid)*100)**2)
print("MSE :", mse)


#loss = np.mean(np.maximum(0,(y_pred - y_valid)-.05) + np.maximum(0,(y_pred - y_valid)-.05))
#print("Loss :", loss)

# Calculer le t_score
train_score = np.mean(np.abs(y_pred_train-y_train)*100<=5)
test_score = np.mean(np.abs(y_pred-y_valid)*100<=5)
print("t_score test :", train_score)
print("t_score train :", test_score)

"""
ids = np.arange(1, len(predictions) + 1)

# Create a DataFrame for the output
output_df = pd.DataFrame({
    'ID': ids,
    'PURITY': predictions
})

# Save the DataFrame to a CSV file
output_df.to_csv('predictions.csv', index=False) 


# Afficher la courbe d'apprentissage
import matplotlib.pyplot as plt
plt.plot(steps, learning_curve)
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.title('Learning Curve')
plt.show()
"""

y_pred contains NaN: False
MSE : 8.301041399378008
t_score test : 0.9461538461538461
t_score train : 0.9038461538461539


/var/folders/93/xgf_b79s5sd1vjn_794f14ym0000gn/T/ipykernel_48663/3560239249.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tensor = torch.tensor(X, dtype=torch.float32)


"\nids = np.arange(1, len(predictions) + 1)\n\n# Create a DataFrame for the output\noutput_df = pd.DataFrame({\n    'ID': ids,\n    'PURITY': predictions\n})\n\n# Save the DataFrame to a CSV file\noutput_df.to_csv('predictions.csv', index=False) \n\n\n# Afficher la courbe d'apprentissage\nimport matplotlib.pyplot as plt\nplt.plot(steps, learning_curve)\nplt.xlabel('Steps')\nplt.ylabel('Loss')\nplt.title('Learning Curve')\nplt.show()\n"

In [95]:
from sklearn.model_selection import LearningCurveDisplay, learning_curve
train_sizes, train_scores, test_scores = learning_curve(nouveau_model, X_train, y_train, train_sizes=np.linspace(0.1, 1.0, 5), cv=5, scoring=custom_scorer)
display = LearningCurveDisplay(train_sizes=train_sizes,
    train_scores=train_scores, test_scores=test_scores, score_name="Score")
display.plot()
plt.show()

ValueError: Found input variables with inconsistent numbers of samples: [1300, 1040]

In [ ]:
torch.save(nouveau_model.model, "model3.pth")

In [68]:
toto = torch.load("model1.pth")
toto.eval()

# Prédiction
predictions = toto(X_train_tensor).flatten().detach().numpy()
print(predictions)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1040x16 and 141x50)